# Churn Model Training + Deployment Pipeline

In [1]:
import mlrun
import pandas as pd

### Setup Project

In [4]:
project = mlrun.get_or_create_project(
    "churn-agents-webinar",
    parameters={
        "base_image" : "mlrun/mlrun:1.7.2",
        "requirements_file" : "requirements-churn-pipeline.txt",
        "force_build" : True, # Disable after running once
        # "source" : "v3io:///bigdata/churn.zip", # enterprise
        "source" : "s3://mlrun/churn.zip" # open source
    }
)

Project Source: v3io:///bigdata/churn.zip
Exporting project as zip archive to v3io:///bigdata/churn.zip...
> 2025-01-27 22:49:52,160 [info] Project loaded successfully: {"project_name":"churn-agents-webinar"}


### Log Dataset

In [5]:
import pandas as pd

dataset = project.log_dataset(key="churn", df=pd.read_csv("data/churn_dataset.csv"))
dataset.uri

'store://datasets/churn-agents-webinar/churn#0@2bba33408a9119f457c80530e2870e06651a5708'

### Run Pipeline

In [6]:
from config import MainConfig

In [7]:
arguments = MainConfig(source_url = dataset.uri).dict()
arguments

{'source_url': 'store://datasets/churn-agents-webinar/churn#0@2bba33408a9119f457c80530e2870e06651a5708',
 'label_column': 'churn',
 'allow_validation_failure': True,
 'test_size': 0.2,
 'model_name': 'churn_model',
 'sentiment_model': 'cardiffnlp/twitter-roberta-base-sentiment-latest',
 'text_column': 'chat_log',
 'sentiment_column': 'sentiment_label',
 'ordinal_columns': ['international_plan', 'voice_mail_plan'],
 'drop_columns': ['area_code', 'chat_log', 'state']}

In [8]:
train_run_id = project.run(
    name="main",
    arguments=arguments,
    dirty=True,
    watch=True
)

> 2025-01-27 22:49:56,063 [warning] WARNING!, You seem to have uncommitted git changes, use .push()
> 2025-01-27 22:49:57,822 [info] Pipeline submitted successfully: {"id":"210aba32-f83a-4c16-8c4a-b1545ff14165","pipeline_name":"churn-agents-webinar-main 2025-01-27 22-49-57"}
> 2025-01-27 22:49:57,823 [info] Pipeline run id=210aba32-f83a-4c16-8c4a-b1545ff14165, check UI for progress


> 2025-01-27 22:49:57,872 [info] Started run workflow churn-agents-webinar-main with run id = '210aba32-f83a-4c16-8c4a-b1545ff14165' by kfp engine
> 2025-01-27 22:49:57,873 [info] Waiting for pipeline run completion: {"project":"churn-agents-webinar","run_id":"210aba32-f83a-4c16-8c4a-b1545ff14165"}


uid,start,state,kind,name,parameters,results
...81fc8f84,Jan 27 22:57:59,completed,run,validate-validate-model,model_path=store://models/churn-agents-webinar/churn_model#1:latest@210aba32-f83a-4c16-8c4a-b1545ff14165label_column=churnallow_validation_failure=True,passed_suite=False
...351c12c2,Jan 27 22:57:06,completed,run,train,label_column=churnmodel_name=churn_model,best_iteration=4accuracy=0.9583333333333334f1_score=0.8571428571428571precision_score=0.868421052631579recall_score=0.8461538461538461model_uri=store://models/churn-agents-webinar/churn_model#1:latest@210aba32-f83a-4c16-8c4a-b1545ff14165
...5aafc720,Jan 27 22:56:35,completed,run,validate-validate-train-test-split,label_column=churnallow_validation_failure=True,passed_suite=True
...e912e5a0,Jan 27 22:56:09,completed,run,data-process-data,"label_column=churntest_size=0.2ordinal_columns=['international_plan', 'voice_mail_plan']drop_columns=['area_code', 'chat_log', 'state']sentiment_column=sentiment_label",
...138e6dda,Jan 27 22:55:18,completed,run,validate-validate-data-integrity,label_column=churnallow_validation_failure=True,passed_suite=False
...3562dc1b,Jan 27 22:51:41,completed,run,data-sentiment-analysis,sentiment_model=cardiffnlp/twitter-roberta-base-sentiment-latesttext_column=chat_log,
...f39ccd3f,Jan 27 22:50:53,completed,run,data-get-data,,


### Test Model Endpoint

In [9]:
serving_fn = project.get_function("serving")

In [10]:
test = mlrun.get_dataitem("store://datasets/churn-agents-webinar/data-process-data_test#0:latest").as_df()
test = test.drop("churn", axis=1)
data = test.head(5).to_dict(orient="split")["data"]

In [11]:
# data

In [14]:
%%time
serving_fn.invoke(
    "/v2/models/churn_model/predict",
    body={
        "inputs": data
    }
)

> 2025-01-27 23:00:31,924 [info] Invoking function: {"method":"POST","path":"http://nuclio-churn-agents-webinar-serving.default-tenant.svc.cluster.local:8080/v2/models/churn_model/predict"}
CPU times: user 14.8 ms, sys: 0 ns, total: 14.8 ms
Wall time: 22.4 ms


{'id': '7a8e3c4b-39cf-4697-a9b0-d661fed881f2',
 'model_name': 'churn_model',
 'outputs': [0.005,
  0.0,
  0.0016666666666666666,
  0.012083333333333333,
  0.008333333333333333],
 'timestamp': '2025-01-27 23:00:31.938748+00:00',
 'model_version': 'latest'}